In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.optimize import curve_fit

# LA Clippers Business Insights & Analytics

In [43]:
# Load the the data from excel file
df_Clippers = pd.read_excel('LA Clippers Data Challenge F&B Dataset.xlsx')

In [44]:
df_Clippers.head()

,OrderID,OrderStatus,CustomerID,ArrivalDatetime,DepartureDatetime,GroupSize,ProductID,ProductName,ProductQuantity,ProductFullPrice,PaymentStatus,OrderTotalQuantity,EmptyOrder,Category,SubCategory
0,1159,paid,1006,2023-03-23 00:13:18,2023-03-23 00:15:06,1,4.453241e+13,Aquafina- 20 oz bottle,1.0,7.15,success,2,No,Non-alcohol,Water
1,1159,paid,1006,2023-03-23 00:13:18,2023-03-23 00:15:06,1,4.453244e+13,Pepperoni Pizza,1.0,18.15,success,2,No,Food,Hot food
2,1160,paid,1001,2023-03-23 00:15:24,2023-03-23 00:17:22,2,4.453244e+13,Starry- 20 oz bottle,1.0,7.15,success,4,No,Non-alcohol,Non-alcohol Beverage
3,1160,paid,1001,2023-03-23 00:15:24,2023-03-23 00:17:22,2,4.453244e+13,Pink's Hot Dog,2.0,18.70,success,4,No,Food,Hot food
4,1160,paid,1001,2023-03-23 00:15:24,2023-03-23 00:17:22,2,4.453244e+13,Sour Patch Candy,1.0,6.05,success,4,No,Food,Snack


In [45]:
# Check the data types of the columns
df_Clippers.dtypes

OrderID                        int64
OrderStatus                   object
CustomerID                     int64
ArrivalDatetime       datetime64[ns]
DepartureDatetime     datetime64[ns]
GroupSize                      int64
ProductID                    float64
ProductName                   object
ProductQuantity              float64
ProductFullPrice             float64
PaymentStatus                 object
OrderTotalQuantity             int64
EmptyOrder                    object
Category                      object
SubCategory                   object
dtype: object

In [47]:
df_Clippers.describe()


,OrderID,CustomerID,ArrivalDatetime,DepartureDatetime,GroupSize,ProductID,ProductQuantity,ProductFullPrice,OrderTotalQuantity
count,65025.000000,65025.000000,65025,65025,65025.000000,6.369500e+04,63695.000000,63695.000000,65025.000000
mean,17931.706344,16208.241753,2023-06-19 10:16:46.851042048,2023-06-19 10:47:01.272941312,1.629727,4.453403e+13,1.169275,14.938188,2.962537
min,1159.000000,9.000000,2023-03-23 00:08:03,2023-03-23 00:15:06,0.000000,4.453240e+13,1.000000,5.500000,0.000000
25%,9499.000000,8551.000000,2023-05-11 00:21:19,2023-05-11 00:23:52,1.000000,4.453241e+13,1.000000,7.150000,2.000000
50%,17866.000000,16164.000000,2023-06-10 02:51:35,2023-06-10 03:16:06,2.000000,4.453243e+13,1.000000,14.300000,3.000000
75%,26054.000000,23691.000000,2023-08-12 00:39:00,2023-08-12 00:43:00,2.000000,4.453244e+13,1.000000,19.810000,4.000000
max,35455.000000,31969.000000,2023-09-04 05:16:01,2023-09-05 03:38:26,9.000000,4.455953e+13,10.000000,203.500000,23.000000
std,9818.958865,8876.491951,NaN,NaN,0.775624,6.112837e+09,0.451269,9.976353,1.823947


In [48]:
# count number of nulls in each column
df_Clippers.isnull().sum()

OrderID                  0
OrderStatus              0
CustomerID               0
ArrivalDatetime          0
DepartureDatetime        0
GroupSize                0
ProductID             1330
ProductName           1330
ProductQuantity       1330
ProductFullPrice      1330
PaymentStatus            0
OrderTotalQuantity       0
EmptyOrder               0
Category              1407
SubCategory           1407
dtype: int64

In [18]:
# select the rows with missing values
df_Clippers[df_Clippers.isnull().any(axis=1)]

,OrderID,OrderStatus,CustomerID,ArrivalDatetime,DepartureDatetime,GroupSize,ProductID,ProductName,ProductQuantity,ProductFullPrice,PaymentStatus,OrderTotalQuantity,EmptyOrder,Category,SubCategory
81,1197,paid,1076,2023-03-23 01:15:21,2023-03-23 01:16:50,2,NaN,NaN,NaN,NaN,success,0,Yes,NaN,NaN
501,1387,paid,1308,2023-03-23 02:13:15,2023-03-23 02:15:13,1,NaN,NaN,NaN,NaN,success,0,Yes,NaN,NaN
553,1416,paid,1337,2023-03-23 02:20:15,2023-03-23 02:22:10,1,NaN,NaN,NaN,NaN,success,0,Yes,NaN,NaN
618,1446,paid,1360,2023-03-23 02:25:48,2023-03-23 02:30:40,2,NaN,NaN,NaN,NaN,success,0,Yes,NaN,NaN
659,1468,paid,1399,2023-03-23 02:35:18,2023-03-23 02:36:52,1,NaN,NaN,NaN,NaN,success,0,Yes,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64991,35436,paid,31962,2023-09-04 05:07:26,2023-09-04 05:09:17,1,NaN,NaN,NaN,NaN,success,0,Yes,NaN,NaN
64992,35437,paid,31969,2023-09-04 05:08:22,2023-09-04 05:09:36,2,NaN,NaN,NaN,NaN,success,0,Yes,NaN,NaN
65009,35446,paid,31750,2023-09-04 02:54:11,2023-09-04 05:36:09,1,NaN,NaN,NaN,NaN,success,0,Yes,NaN,NaN
65016,35451,paid,31800,2023-09-04 03:05:53,2023-09-04 05:36:43,1,NaN,NaN,NaN,NaN,success,0,Yes,NaN,NaN


In [50]:
df_Clippers[(df_Clippers['EmptyOrder'] == 'Yes')].shape[0]

1330

In [49]:
# count number of rows with EmptyOrder = Yes and ProcutID = NaN 
df_Clippers[(df_Clippers['EmptyOrder'] == 'Yes') & (df_Clippers['ProductID'].isnull())].shape[0]

1330

Here I identified that all the ProductID with null values are linked with EmptyOrder == Yes. I assume that this colum was created to identify missing values in the product category. To proceed with the analysis, I decided to separate this data from the one with values for simplicity. An important note is that is a good practice to save the information with NaN values so as to healp identifing why the EmptyOrders are created (inflo like OrderID, Timestamp, paid or not etc).

In [54]:
# separate the rows with EmptyOrder = Yes 
df_Clippers_EmptyOrder = df_Clippers[(df_Clippers['EmptyOrder'] == 'Yes')]
print(df_Clippers_EmptyOrder.shape[0])

# separate the rows with EmptyOrder = No
df_Clippers_complete = df_Clippers[(df_Clippers['EmptyOrder'] == 'No')]
print(df_Clippers_complete.shape[0])

1330
63695


In [55]:
df_Clippers_complete.isnull().sum()

OrderID                0
OrderStatus            0
CustomerID             0
ArrivalDatetime        0
DepartureDatetime      0
GroupSize              0
ProductID              0
ProductName            0
ProductQuantity        0
ProductFullPrice       0
PaymentStatus          0
OrderTotalQuantity     0
EmptyOrder             0
Category              77
SubCategory           77
dtype: int64

However, there are 77 products that were actually bought but are not assigned to a `Category` and `SubCategory`. Now I'll identified what product are those to see if I can complete them with the right value:

In [56]:
# select the rows with Empty order = No and missing values in the colum category
df_Clippers_complete['ProductName'][(df_Clippers_complete['EmptyOrder'] == 'No') & (df_Clippers_complete['Category'].isnull())].value_counts()

ProductName
Ciroc Spritz Colada    77
Name: count, dtype: int64

In [57]:
df_Clippers_complete.value_counts('Category')

Category
Food           23715
Alcohol        22445
Non-alcohol    17458
Name: count, dtype: int64

In [58]:
df_Clippers_complete[df_Clippers_complete['Category'] == 'Alcohol'].value_counts('SubCategory')

SubCategory
Beer      11246
Liquor    11199
Name: count, dtype: int64

In [59]:
# select producnt name with subcategory = Liquor
df_Clippers_complete['ProductName'][df_Clippers_complete['SubCategory'] == 'Liquor'].value_counts()

ProductName
White Claw Mango 24 oz can             2477
White Claw Black Cherry 24 oz can      2184
Cutwater Mango Margarita- 12 oz can    1693
Cutwater Lime Margarita- 12 oz can     1351
Ciroc Spritz Watermelon kiwi           1026
Ciroc Spritz Pineapple Passion          750
Ciroc spritz Citrus Sunset              678
Cutwater Mai Tai- 12 oz can             419
Cutwater Vodka Mule- 12 oz can          367
Ciroc spritz Colada                     254
Name: count, dtype: int64

Here I identify that the Ciroc Spritz Colada is actually classified as category 'Alcohol' and subcategory 'Liquor'

In [64]:
# fill the missing values in the column category with the value 'Alcohol', subcategory='Liquor' for the rows with product name = 'Ciroc spritz Colada' 
df_Clippers_complete.loc[df_Clippers_complete['ProductName'] == 'Ciroc Spritz Colada', 'Category'] = 'Alcohol'
df_Clippers_complete.loc[df_Clippers_complete['ProductName'] == 'Ciroc Spritz Colada', 'SubCategory'] = 'Liquor'

In [65]:
df_Clippers_complete['ProductName'][(df_Clippers_complete['EmptyOrder'] == 'No') & (df_Clippers_complete['Category'].isnull())].value_counts()

Series([], Name: count, dtype: int64)

In [66]:
df_Clippers_complete['ProductName'][(df_Clippers_complete['EmptyOrder'] == 'No') & (df_Clippers_complete['SubCategory'].isnull())].value_counts()

Series([], Name: count, dtype: int64)

In [67]:
# count number of nulls in each column
df_Clippers_complete.isnull().sum()

OrderID               0
OrderStatus           0
CustomerID            0
ArrivalDatetime       0
DepartureDatetime     0
GroupSize             0
ProductID             0
ProductName           0
ProductQuantity       0
ProductFullPrice      0
PaymentStatus         0
OrderTotalQuantity    0
EmptyOrder            0
Category              0
SubCategory           0
dtype: int64

Therefore, I can assume that the final data is clean and ready to use with the right values.

In [68]:
df_Clippers_complete.dtypes

OrderID                        int64
OrderStatus                   object
CustomerID                     int64
ArrivalDatetime       datetime64[ns]
DepartureDatetime     datetime64[ns]
GroupSize                      int64
ProductID                    float64
ProductName                   object
ProductQuantity              float64
ProductFullPrice             float64
PaymentStatus                 object
OrderTotalQuantity             int64
EmptyOrder                    object
Category                      object
SubCategory                   object
dtype: object

In [69]:
# now I'll change ProductID type to int64 for better data management 
df_Clippers_complete['ProductID'] = df_Clippers_complete['ProductID'].astype('int64')

df_Clippers_complete.dtypes

C:\Users\Facundo\AppData\Local\Temp\ipykernel_4036\1365318354.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Clippers_complete['ProductID'] = df_Clippers_complete['ProductID'].astype('int64')


OrderID                        int64
OrderStatus                   object
CustomerID                     int64
ArrivalDatetime       datetime64[ns]
DepartureDatetime     datetime64[ns]
GroupSize                      int64
ProductID                      int64
ProductName                   object
ProductQuantity              float64
ProductFullPrice             float64
PaymentStatus                 object
OrderTotalQuantity             int64
EmptyOrder                    object
Category                      object
SubCategory                   object
dtype: object

Deliverable 1: 

In [115]:
df_Clippers_complete[df_Clippers_complete.OrderID == 1160]

,OrderID,OrderStatus,CustomerID,ArrivalDatetime,DepartureDatetime,GroupSize,ProductID,ProductName,ProductQuantity,ProductFullPrice,PaymentStatus,OrderTotalQuantity,EmptyOrder,Category,SubCategory
2,1160,paid,1001,2023-03-23 00:15:24,2023-03-23 00:17:22,2,44532435157305,Starry- 20 oz bottle,1.0,7.15,success,4,No,Non-alcohol,Non-alcohol Beverage
3,1160,paid,1001,2023-03-23 00:15:24,2023-03-23 00:17:22,2,44532436828473,Pink's Hot Dog,2.0,18.70,success,4,No,Food,Hot food
4,1160,paid,1001,2023-03-23 00:15:24,2023-03-23 00:17:22,2,44532439187769,Sour Patch Candy,1.0,6.05,success,4,No,Food,Snack


In [70]:
from pandasql import sqldf 
import pandas as pd 

pysqldf = lambda q: sqldf(q)
pysqldf("SELECT * FROM df_Clippers_complete")


,OrderID,OrderStatus,CustomerID,ArrivalDatetime,DepartureDatetime,GroupSize,ProductID,ProductName,ProductQuantity,ProductFullPrice,PaymentStatus,OrderTotalQuantity,EmptyOrder,Category,SubCategory
0,1159,paid,1006,2023-03-23 00:13:18.000000,2023-03-23 00:15:06.000000,1,44532414808377,Aquafina- 20 oz bottle,1.0,7.15,success,2,No,Non-alcohol,Water
1,1159,paid,1006,2023-03-23 00:13:18.000000,2023-03-23 00:15:06.000000,1,44532437975353,Pepperoni Pizza,1.0,18.15,success,2,No,Food,Hot food
2,1160,paid,1001,2023-03-23 00:15:24.000000,2023-03-23 00:17:22.000000,2,44532435157305,Starry- 20 oz bottle,1.0,7.15,success,4,No,Non-alcohol,Non-alcohol Beverage
3,1160,paid,1001,2023-03-23 00:15:24.000000,2023-03-23 00:17:22.000000,2,44532436828473,Pink's Hot Dog,2.0,18.70,success,4,No,Food,Hot food
4,1160,paid,1001,2023-03-23 00:15:24.000000,2023-03-23 00:17:22.000000,2,44532439187769,Sour Patch Candy,1.0,6.05,success,4,No,Food,Snack
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63690,35453,paid,31597,2023-09-04 02:18:28.000000,2023-09-04 14:37:14.000000,4,44532435943737,Ciroc Spritz Watermelon kiwi,1.0,19.25,success,4,No,Alcohol,Liquor
63691,35453,paid,31597,2023-09-04 02:18:28.000000,2023-09-04 14:37:14.000000,4,44532434469177,Lagunita IPA 24 oz can,1.0,20.91,success,4,No,Alcohol,Beer
63692,35453,paid,31597,2023-09-04 02:18:28.000000,2023-09-04 14:37:14.000000,4,44532435779897,Ciroc Spritz Pineapple Passion,1.0,19.25,success,4,No,Alcohol,Liquor
63693,35453,paid,31597,2023-09-04 02:18:28.000000,2023-09-04 14:37:14.000000,4,44532435878201,Ciroc spritz Citrus Sunset,1.0,19.25,success,4,No,Alcohol,Liquor


In [133]:
# identify if there are duplicate rows in the data
df_Clippers_complete.duplicated().sum()

0

In [134]:
# download the two dataframes to excel files
df_Clippers_complete.to_excel('df_Clippers_complete.xlsx', index=False)
df_Clippers_EmptyOrder.to_excel('df_Clippers_EmptyOrder.xlsx', index=False)


#### a.	[SQL] What is the average spending for a group of 2 people? 

In [136]:
pysqldf("""SELECT 
                avg(ProductFullPrice) as Average_spending_group_of2
            FROM df_Clippers_complete 
            WHERE GroupSize = 2""")

,Average_spending_group_of2
0,15.357059


In [ ]:
average = pysqldf("""SELECT 
                    avg(ProductFullPrice) 
                  FROM df_Clippers_complete 
                  WHERE GroupSize = 2""")

In [83]:
average

,avg(ProductFullPrice)
0,15.357059


In [87]:
print("The average price of ticket with group size = 2 is: USD", round(average.iloc[0,0],2))

The average price of ticket with group size = 2 is: USD 15.36


#### b.	[SQL] What was the highest amount of Food purchased in a day and what was the date (please report on ArrivalDatetime in PDT)?  

Since pandasql runs with SQLite under the hood, I have to use the datetime function to manualy substract 7 hours from the UTC format to convert arrivaldatetime into PDT. This is not an automatization, because the time differences change from winter to summer, but is a sumplification for this problem. Note that other SQL versions (like mysql) do support timezone functions to automatize this process and prevent ourselves from errors.

In [108]:
pysqldf("""SELECT 
                date(datetime(ArrivalDatetime, '-7 hours')) AS pdt_time
                ,sum(productquantity) as total_quantity
            FROM df_Clippers_complete 
            WHERE Category = 'Food'
            GROUP BY pdt_time
            ORDER BY total_quantity desc
            LIMIT 1""")

,pdt_time,total_quantity
0,2023-08-26,1186.0


#### c.	[SQL] What were the top 5 spending orders? The result should have 4 columns: OrderID, NumOfProducts, TotalRevenue, Rank. Sort by total revenue descending.

In [130]:
pysqldf("""SELECT 
                OrderID
                ,sum(ProductQuantity) as NumOfProducts
                ,sum(ProductFullPrice) as TotalRevenue
                ,rank() OVER (ORDER BY sum(ProductFullPrice) desc) as Rank
            FROM df_Clippers_complete 
            GROUP BY OrderID
            ORDER BY TotalRevenue desc
            LIMIT 5
            """)

,OrderID,NumOfProducts,TotalRevenue,Rank
0,18668,14.0,282.70,1
1,16197,14.0,264.56,2
2,20940,21.0,215.21,3
3,26736,16.0,212.77,4
4,30251,10.0,200.77,5
